In [ ]:
# Calculate std
# Remove value that are lower than std
# Calculate new std
# Calculate new mean
# > mean + 2*std -> attention
# > mean + 4*std -> alert
# > mean + 8*std -> danger!

In [ ]:
high_value_counts_np = np.array(high_value_counts)
higher_std = high_value_counts_np[high_value_counts_np > np.std(high_value_counts)]
threshold = higher_std.mean() + higher_std.std()

for i, (fn, val) in enumerate(zip(selected_source_list, high_value_counts)):
    if val > threshold:
        print(f'{i} {fn} {val}')
        